In [24]:
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [25]:
labels = pd.read_csv("Xray_TeethLabels_Simple.csv",index_col=0)
Ylabels = labels[5:]
metaY = labels[:3]
X = pd.DataFrame()
Y = []
for picFilename in glob.glob("processed3/processed/*"):
    #print(picFilename)
    patNumber = picFilename.split('/')[2].split('_')[0]
    #toothNumber = picFilename.split('/')[2].split('_')[1].split('.')[0]
    toothNumber1 = picFilename.split('/')[2].split('_')[1]
    toothNumber2 = picFilename.split('/')[2].split('_')[2].split('.')[0]
    im = Image.open(picFilename)
    #X[patNumber + '_' + toothNumber] = np.array(im).flatten()
    #Y.append(Ylabels.loc[toothNumber,patNumber])
    
    X[patNumber + '_' + toothNumber1 + '_' + toothNumber2] = np.array(im).flatten()
    if Ylabels.loc[toothNumber1,patNumber] == 'Yes' or Ylabels.loc[toothNumber2,patNumber] == 'Yes':
        Y.append(1)
    else:
        Y.append(0)

X_train, X_test, y_train, y_test = train_test_split(X.T.to_numpy(), np.array(Y), test_size=0.20, random_state=100)

In [26]:
np.random.seed(100)
def svc_param_selection(X, y, nfolds):
    C = [0.1, 1, 10, 100, 1000] #try different values for C
    kernel = ['linear', 'rbf']
    param_grid = {'C': C, 'kernel': kernel}
    grid_search = GridSearchCV(SVC(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_
best_c = svc_param_selection(X.T.to_numpy(), np.array(Y), 2)
print ("The best value for C is:",best_c)

The best value for C is: {'C': 0.1, 'kernel': 'linear'}


In [27]:
model = SVC(C=best_c['C'], kernel=best_c['kernel'])
model.fit(X_train, y_train)
print(model)
# make predictions
expected = y_test
predicted = model.predict(X_test)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

SVC(C=0.1, kernel='linear')
              precision    recall  f1-score   support

           0       0.71      0.67      0.69        15
           1       0.75      0.79      0.77        19

    accuracy                           0.74        34
   macro avg       0.73      0.73      0.73        34
weighted avg       0.73      0.74      0.73        34

[[10  5]
 [ 4 15]]


In [28]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test, predicted).ravel()
print(tn)
print(tp)
print(fp)
print(fn)
print(f"sensitivity: {tp/(tp+fn)}")
print(f"Specificity: {tn/(tn+fp)}")
print(f"FPR: {fp/(fp+tn)}")
print(f" FNR: {fn/(fn+tp)}")
print(f"Accuracy: {(tp+tn)/(tp+tn+fp+fn)}")
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print(f"precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {(2*precision*recall)/(precision+recall)}")

10
15
5
4
sensitivity: 0.7894736842105263
Specificity: 0.6666666666666666
FPR: 0.3333333333333333
 FNR: 0.21052631578947367
Accuracy: 0.7352941176470589
precision: 0.75
Recall: 0.7894736842105263
F1 score: 0.7692307692307692
